<a href="https://colab.research.google.com/github/marinaceccon1/LearningFromNetworksProject/blob/main/Degree_ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Computation of the first approach with the degree centrality

In [ ]:
#import of the needed libraries
import networkx as nx
import numpy as np
import sys
import collections
from builtins import sum as siumm

In [ ]:
#clone the github repository containing the dataset
!git clone https://github.com/marinaceccon1/LearningFromNetworksProject.git

Cloning into 'LearningFromNetworksProject'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 18 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), 6.19 MiB | 2.51 MiB/s, done.


In [ ]:
# parameter K for the K nodes with the correct ratio centr/lcc
K = 12000

In [ ]:
# load the data and create the graph representing the network
matrix = np.loadtxt('/content/LearningFromNetworksProject/out.txt', usecols=range(2))
G = nx.from_edgelist(matrix)

In [ ]:
# computes the degree centrality and order the dictionary w.r.t. the keys
deg_centralities = nx.degree_centrality(G)
ordered_deg_centralities = collections.OrderedDict(sorted(deg_centralities.items()))

In [ ]:
# computes the local clustering coefficient and order the dictionary w.r.t. the keys
lcc = nx.clustering(G)
ordered_lcc = collections.OrderedDict(sorted(lcc.items()))

In [ ]:
#define a softmax function
def softmax(vector):
	e = np.exp(vector)
	return e / e.sum()

In [ ]:
# extract the values of degree centrality in the dictionary and insert them in a list 
ordered_deg_centr_values = list(ordered_deg_centralities.values())

# extract the values of local clustering coefficient in the dictionary and insert them in a list 
# replace the vector with its softmax in order to avoid zero values
ordered_lcc_values = softmax(list(ordered_lcc.values())) 

In [ ]:
# check if everything is ok
assert[len(ordered_deg_centr_values)==len(ordered_lcc_values)]

ratios = {} #new dictionary containing as value deg_centr/lcc

for i in range(len(ordered_deg_centr_values)):
  ratios[i+1] = (ordered_deg_centr_values[i]/ordered_lcc_values[i]) #the keys in the dictionaries start from 1, the indices in the lists start from 0

# sort the dictionary by value, since reverse is equal to True, sort them in decreasing order
# in this way the first K nodes are the one we are interested in
sorted_ratios = sorted(ratios.items(), key=lambda x:x[1], reverse = True) 

# the output of the above function is a list of pairs of numbers, in each pair the first number represents 
# the key of the node, the second number is its value ratio = deg_centr/lcc
# to access the key of the i-th pair the necessary command is sorted_ratio[i][0]

In [ ]:
#sorted_ratios is a list of pairs of numbers, in each pair the first number represents the key of the node, 
#the second number is its value ratio = deg_centr/lcc
#to access the key of the i-th pair the necessary command is sorted_ratios[i][0]

cardinalities = {} #new dictionary, the keys will be pairs of nodes that are not adjacent, the value is the cardinality 
#of the intersection of their neighbourhoods

for i in range(K): #approximation: we only consider the K nodes with highest ratio
  for j in range(i+1,K): #in order not to contemplate the same pair twice, first as (i,j) and after as (j,i), we set j>i
    
    breakout_flag = False #needed to compute a double break
    for key,value in (G.adj[sorted_ratios[i][0]]).items(): #check the neighbours of sorted_ratios[i][0]      
      if key == sorted_ratios[j][0]: #if inverse_sorted_ratio[j][0] is a neighbour of sorted_ratios[i][0]
        breakout_flag = True
        break #exit the for key,value loop
    if breakout_flag:
      break #exit the for j loop

    s1 = set(G.adj[sorted_ratios[i][0]]) #convert the dictionary in a set containing all the keys in the dictionary
    s2 = set(G.adj[sorted_ratios[j][0]])
    res = s1 & s2 #keys that belong to both sets

    cardinalities[(sorted_ratios[i][0], sorted_ratios[j][0])] = len(res) #the value associated to that pair 
    #of nodes is the cardinality of the intersection computed above

inverse_sorted_card = sorted(cardinalities.items(), key=lambda x:x[1], reverse = True) #order the dictionary by value

In [ ]:
N = 500

In [ ]:
G_i_deg = G.copy()

In [ ]:
from builtins import sum as siumm
#add the first N edges of inverse_sorted_card
for i in range(N):
  u_deg = inverse_sorted_card[i][0][0]
  v_deg = inverse_sorted_card[i][0][1]
  G_i_deg.add_edge(u_deg,v_deg)

glo_clu_coeff_deg = nx.transitivity(G_i_deg) #compute the global clustering coefficient of the new graph
print(glo_clu_coeff_deg)

0.14818319804572128


In [ ]:
triangles_tot = siumm(nx.triangles(G_i_deg).values()) #compute the number of triangles of the new graph
print(triangles_tot/3)

3521374.0


# Computation of the number of triangles after adding N random edges to G

In [ ]:
import random
from builtins import sum as siumm

N=500
glo_clu=[]
triangles = []
#average 10 clustering coefficents you get by adding N random edges 
for i in range(10):
  G_i = G.copy()
  j = 1
  while j <= N:
    u = random.randint(1,G.number_of_nodes())
    v = random.randint(1,G.number_of_nodes())
    if not G_i.has_edge(u, v):
      G_i.add_edge(u,v)
      j+=1
  triangle = siumm(nx.triangles(G_i).values())/3
  triangles.append(triangle)
avg_triangl = sum(triangles)/len(triangles)